___
# Ciência dos Dados - PROJETO 2

___
## Nome 1
#### Gabriel Couto

## Nome 2
#### Gabriel Miras

## Nome 3
#### Mariana Abrantes

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [1]:
import pandas as pd
import os
import random
import numpy as np

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gabri\Desktop\Insper Semestre 2DP\Ciências dos Dados\CDadosDP-P2 (Miras e e Mari)


In [3]:
leitura = pd.ExcelFile('spamham2019(1).xlsx')
dados = pd.read_excel(leitura)
dados.head()

,Email,Class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [4]:
#Separar tabela em treino e teste:
indices = range(len(dados))
indTreino = random.sample(indices, int(len(indices)*0.75))
indTeste = []
for i in indices:
    if i not in indTreino:
        indTeste.append(i)

treino = dados.loc[indTreino,:]
teste = dados.loc[indTeste,:]

#Cria índices numéricos pros dados
treino.index = range(len(treino))
teste.index = range(len(teste))

# Limpando a base de dados

In [5]:
tirar = ['!','@','#','$','%','¨','&','*','(',')','-','_','+','=','§',',','.','<','>',':',';','/','?','°','|','\\','{','}','ª','[',']','º','×','–','—','•','…','‹','›','€','1','2','3','4','5','6','7','8','9','0']
for i in tirar:
    treino.Email = treino.Email.str.replace(i, '', regex=False)
    teste.Email = teste.Email.str.replace(i, '', regex=False)

In [6]:
#CRIAR TABELA DE FREQUENCIA DE CADA PALAVRA
freq = {} #Tabela de frequências 
#Dicionario tabela de frequências:
    #   'H' = nº de vezes que determinada palavra aparece em email Ham
    #   'S' = nº de vezes que determinada palavra aparece em email Spam 
    #'pPiH' = P(P∩R) = prob de determinada palavra aparecer num email Ham
    #'pPiS' = P(P∩S) = prob de determinada palavra aparecer num email spam
    #'pPcH' = P(P|H) = prob de determinada palavra aparecer dado que o email é Ham
    #'pPcS' = P(P|S) = prob de determinada palavra aparecer dado que o email é Spam

for numLinha in range(len(treino.Email)): 
    palavras_linha = treino.loc[numLinha,'Email'].split(' ')
    for palavra in palavras_linha:
        if palavra not in freq:
            freq[palavra] = {'H':0,'S':0,'pPiH':0,'pPiS':0,'pPcH':0,'pPcS':0}
        if treino.loc[numLinha,'Class'] == 'ham':
            freq[palavra]['H'] +=1
        else:
            freq[palavra]['S'] +=1

In [41]:
#PH = Prob do email ser Ham | PS = Prob email ser Spam
PH = len(treino[treino['Class']=='ham'])/len(treino)
PS = len(treino[treino['Class']=='spam'])/len(treino)

palavrasH,palavrasS = 0,0 #total de palavras em emails HAM e total de palavras em emails Spam (contando repetições das palavras)
for key,valor in freq.items():  
    palavrasH += valor['H']
    palavrasS += valor['S']
totalPalavras = palavrasH + palavrasS

for palavra,valor in freq.items():
    #Prob de determinada palavra aparecer:
    pP = (valor['H']+valor['S'])/totalPalavras
    
    #adicionamos 1 ao numerador para que a probabilidade nunca dê zero
    freq[palavra]['pP'] = pP
    freq[palavra]['pPiH'] = (valor['H']+1)/(totalPalavras+1) #(2*totalR+totalI)
    freq[palavra]['pPiS'] = (valor['S']+1)/(totalPalavras+1) #/(2*totalI+totalR)
    
    #freq[palavra]['pPcH'] = freq[palavra]['pPiH']/PH
    #freq[palavra]['pPcS'] = freq[palavra]['pPiS']/PS

In [42]:
print('PH:',PH)
freq['Its']

PH: 0.8683895668820292


{'H': 55,
 'S': 1,
 'pPiH': 0.0008638909029202598,
 'pPiS': 3.085324653286642e-05,
 'pPcH': 0.0009771001180197676,
 'pPcS': 0.00011721971333202435,
 'pP': 0.0008639042300453549}

In [43]:
freq['Its']['pPiH']+freq['Its']['pPiS'] == freq['Its']['pP']

False

___
## 3. Classificador Naive-Bayes

In [46]:
mensagens={}

for numLinha in range(len(teste.Email)): 
    palavras_linha = teste.loc[numLinha,'Email'].split(' ')
    mensagens[numLinha] = {'Email':teste.loc[numLinha,'Email']}
    pLH,pLS =1,1    
    
    for palavra in palavras_linha:       
        if palavra in freq:
            pLH*= freq[palavra]['pPiH']/freq[palavra]['pP']
            pLS*= freq[palavra]['pPiS']/freq[palavra]['pP']
    
    mensagens[numLinha]['pLH'] = pLH
    mensagens[numLinha]['pLS'] = pLS
    
    if pLS > pLH:
        mensagens[numLinha]['Classe'] = 'Spam'
    else:
        mensagens[numLinha]['Classe'] = 'Ham'
        
        #E se a palavra não tiver na tabela de freq finge que ela n existe?

            
mensagens

{0: {'Email': 'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat',
  'pLH': 1.3576839570407688,
  'pLS': 6.026887781325791e-15,
  'Classe': 'Ham'},
 1: {'Email': "FreeMsg Hey there darling it's been  week's now and no word back I'd like some fun you up for it still Tb ok XxX std chgs to send å£ to rcv",
  'pLH': 2.4857604790662806e-06,
  'pLS': 5.547228185618426e-19,
  'Classe': 'Ham'},
 2: {'Email': 'SIX chances to win CASH From  to  pounds txt CSH and send to  Cost pday days  TsandCs apply Reply HL  info',
  'pLH': 3.497119272956393e-10,
  'pLS': 5.4143218522907146e-05,
  'Classe': 'Spam'},
 3: {'Email': "I've been searching for the right words to thank you for this breather I promise i wont take your help for granted and will fulfil my promise You have been wonderful and a blessing at all times",
  'pLH': 0.02691775021446009,
  'pLS': 6.991374265988959e-30,
  'Classe': 'Ham'},
 4: {'Email': 'I HAVE A DATE ON SUNDAY WITH WILL',
  '

___
## 4. Qualidade do Classificador alterando a base de treinamento